In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\shash\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shash\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shash\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shash\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
from keras.models import load_model

IMAGE_SIZE = [224, 224]

train_path = r'S:\VIT AP\SummerInternship1\COVID 19\train'
valid_path = r'S:\VIT AP\SummerInternship1\COVID 19\test'

In [3]:
# add preprocessing layer
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False

In [4]:
# layers
x = Flatten()(resnet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(7, activation='softmax')(x)

In [5]:
# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [6]:
# view the structure of the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
# telling the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [8]:
#resizing the images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
training_set = train_datagen.flow_from_directory(r'S:\VIT AP\SummerInternship1\COVID 19\train',
                                                 target_size = (224, 224),
                                                 batch_size = 1,
                                                 class_mode = 'categorical')

Found 289 images belonging to 7 classes.


In [12]:
test_set = test_datagen.flow_from_directory(r'S:\VIT AP\SummerInternship1\COVID 19\test',
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical')

Found 76 images belonging to 7 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=500,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/500
289/289 [==============================] - 105s 365ms/step - loss: 13.1664 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 2/500
289/289 [==============================] - 80s 276ms/step - loss: 13.1622 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 3/500
289/289 [==============================] - 83s 288ms/step - loss: 13.1622 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 4/500
289/289 [==============================] - 88s 304ms/step - loss: 13.1622 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 5/500
289/289 [==============================] - 90s 311ms/step - loss: 13.1622 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 6/500
289/289 [==============================] - 97s 337ms/step - loss: 13.1622 - acc: 0.1834 - val_loss: 13.3611 - val_acc: 0.1711
Epoch 7/500
289/289 [==============================] - 94s 325ms/step - loss:

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf
from keras.models import load_model

In [ ]:
model.save('CovidChest2.h5')